<a href="https://colab.research.google.com/github/Viditha2277/weather_notes/blob/main/proposed1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # For Linear Algebra
import pandas as pd # To Work With Datafor visualizations
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime # Time Series analysis.

In [2]:
df = pd.read_csv("Weather.csv")
df.head()

,Unnamed: 0,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,0,1901,17.99,19.43,23.49,26.41,28.28,28.60,27.49,26.98,26.26,25.08,21.73,18.95
1,1,1902,19.00,20.39,24.10,26.54,28.68,28.44,27.29,27.05,25.95,24.37,21.33,18.78
2,2,1903,18.32,19.79,22.46,26.03,27.93,28.41,28.04,26.63,26.34,24.57,20.96,18.29
3,3,1904,17.77,19.39,22.95,26.73,27.83,27.85,26.84,26.73,25.84,24.36,21.07,18.84
4,4,1905,17.40,17.79,21.78,24.84,28.32,28.69,27.67,27.47,26.29,26.16,22.07,18.71


In [3]:
df = pd.read_csv("Weather.csv", index_col=0)
df1 = pd.melt(df, id_vars='YEAR', value_vars=df.columns[1:]) ## This will melt the data
df1.head() 

,YEAR,variable,value
0,1901,JAN,17.99
1,1902,JAN,19.00
2,1903,JAN,18.32
3,1904,JAN,17.77
4,1905,JAN,17.40


In [4]:
df1['Date'] = df1['variable'] + ' ' + df1['YEAR'].astype(str)  
df1.loc[:,'Date'] = df1['Date'].apply(lambda x : datetime.strptime(x, '%b %Y')) ## Converting String to datetime object
df1.head()

,YEAR,variable,value,Date
0,1901,JAN,17.99,1901-01-01
1,1902,JAN,19.00,1902-01-01
2,1903,JAN,18.32,1903-01-01
3,1904,JAN,17.77,1904-01-01
4,1905,JAN,17.40,1905-01-01


In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 

df2 = df1[['YEAR', 'variable', 'value']].copy()
df2 = pd.get_dummies(df2)
y = df2[['value']]
x = df2.drop(columns='value')

dtr = DecisionTreeRegressor()
train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.3)
dtr.fit(train_x, train_y)
pred = dtr.predict(test_x)
r2_score(test_y, pred)

0.9594224000023633

In [6]:
next_Year = df1[df1['YEAR']==2017][['YEAR', 'variable']]
next_Year.YEAR.replace(2017,2018, inplace=True)
next_Year= pd.get_dummies(next_Year)
temp_2018 = dtr.predict(next_Year)

temp_2018 = {'variable':df1['variable'].unique(), 'value':temp_2018}
temp_2018=pd.DataFrame(temp_2018)
temp_2018['YEAR'] = 2018
temp_2018

,variable,value,YEAR
0,JAN,20.92,2018
1,FEB,23.08,2018
2,MAR,26.61,2018
3,APR,29.17,2018
4,MAY,30.47,2018
5,JUN,29.44,2018
6,JUL,28.31,2018
7,AUG,28.12,2018
8,SEP,28.11,2018
9,OCT,27.24,2018


In [7]:
forecasted_temp = pd.concat([df1,temp_2018], sort=False).groupby(by='YEAR')['value'].mean().reset_index()
fig = go.Figure(data=[
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp['YEAR'], y=forecasted_temp['value'], mode='lines'),
    go.Scatter(name='Yearly Mean Temprature', x=forecasted_temp ['YEAR'], y=forecasted_temp['value'], mode='markers')
])
fig.update_layout(title='Forecasted Temprature:',
                 xaxis_title='Time', yaxis_title='Temprature in Degrees')
fig.show()